In [1]:
! pip install flask


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

from flask import Flask, jsonify, request
import os
import glob
import random
import base64

In [3]:
app = Flask(__name__)

In [4]:

FOLDERS = {
    'Руки': r"C:\Users\user\Desktop\fitness_итог\hands",
    'Спина': r"C:\Users\user\Desktop\fitness_итог\back",
    'Пресс': r"C:\Users\user\Desktop\fitness_итог\press",
    'Грудь': r"C:\Users\user\Desktop\fitness_итог\breast",
    'Ноги': r"C:\Users\user\Desktop\fitness_итог\legs",
    'Плечи': r"C:\Users\user\Desktop\fitness_итог\shoulders"
}

#загрузка файлов
files_found = {}
for name, path in FOLDERS.items():
    if os.path.exists(path):
        gifs = glob.glob(os.path.join(path, "*.gif"))
        files_found[name] = gifs
    else:
        files_found[name] = []

In [5]:
#классификатор
def get_muscle_group(query):
    query_lower = query.lower()
    
    if 'груд' in query_lower or 'жим' in query_lower:
        return 'Грудь'
    elif 'спин' in query_lower or 'тяга' in query_lower:
        return 'Спина'
    elif 'ног' in query_lower or 'присед' in query_lower:
        return 'Ноги'
    elif 'бицепс' in query_lower or 'трицепс' in query_lower or 'руки' in query_lower:
        return 'Руки'
    elif 'плеч' in query_lower or 'дельт' in query_lower:
        return 'Плечи'
    elif 'пресс' in query_lower or 'живот' in query_lower:
        return 'Пресс'
    else:
        for group, files in files_found.items():
            if files:
                return group
        return 'Грудь'

In [6]:

@app.route('/api/predict', methods=['POST', 'GET', 'OPTIONS'])
def predict():
    if request.method == 'OPTIONS':
        response = jsonify({"status": "ok"})
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type')
        response.headers.add('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
        return response
    
    try:
        if request.method == 'GET':
            query = request.args.get('query', '')
        else:
            data = request.get_json(force=True, silent=True)
            if not data:
                return jsonify({"error": "Нет данных JSON", "success": False}), 400
            query = data.get('query', '')
        
        query = query.strip()
        
        if not query:
            return jsonify({"error": "Пустой запрос", "success": False}), 400
        
        muscle_group = get_muscle_group(query)
        
        gif_data = None
        filename = None
        
        if muscle_group in files_found and files_found[muscle_group]:
            files = files_found[muscle_group]
            if files:
                filepath = random.choice(files)
                filename = os.path.basename(filepath)
                
                try:
                    with open(filepath, 'rb') as f:
                        file_bytes = f.read()
                    
                    file_b64 = base64.b64encode(file_bytes).decode('utf-8')
                    gif_data = f"data:image/gif;base64,{file_b64}"
                    
                except Exception as e:
                    gif_data = None
        
        result = {
            "success": True,
            "query": query,
            "muscle_group": muscle_group,
            "gif_found": gif_data is not None,
            "gif_data": gif_data,
            "filename": filename
        }
        
        response = jsonify(result)
        response.headers.add('Access-Control-Allow-Origin', '*')
        return response
        
    except Exception as e:
        error_response = jsonify({
            "error": str(e),
            "success": False
        })
        error_response.headers.add('Access-Control-Allow-Origin', '*')
        return error_response, 500


    

In [8]:
# Запуск сервера
if __name__ == '__main__':
    app.run(
        host='0.0.0.0',
        port=5000,
        debug=True,
        use_reloader=False
    )

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.31.113:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2025 13:02:25] "OPTIONS /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:25] "OPTIONS /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:25] "POST /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:25] "POST /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:29] "OPTIONS /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:29] "POST /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:32] "OPTIONS /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:02:32] "POST /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:07:50] "OPTIONS /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:07:50] "OPTIONS /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:07:50] "POST /api/predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2025 13:07:50] "POST /api/predic